## Домашнее задание 5

In [40]:
import numpy as np
import os
import pandas as pd

In [41]:
from IPython.display import display, clear_output

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

## Get data

In [42]:
interactions_df = pd.read_csv('../data_original/interactions.csv')
users_df = pd.read_csv('../data_original/users.csv')
items_df = pd.read_csv('../data_original/items.csv')

In [43]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [44]:
interactions_df = interactions_df[interactions_df['last_watch_dt'] < '2021-04-01']

In [45]:
interactions_df.shape

(263874, 5)

In [46]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'item_id']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 86614
# users with at least 5 interactions: 14563


In [47]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df,
               how = 'right',
               left_on = 'user_id',
               right_on = 'user_id')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 263874
# of interactions from users with at least 5 interactions: 142670


## Preprocessing

In [48]:
import math

In [49]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'item_id'])['watched_pct'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 142670


,user_id,item_id,watched_pct
0,21,849,6.375039
1,21,4345,6.658211
2,21,10283,6.658211
3,21,12261,6.658211
4,21,15997,6.658211
5,32,952,6.044394
6,32,4382,4.954196
7,32,4807,6.658211
8,32,10436,6.658211
9,32,12132,6.658211


In [50]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'],
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 114136
# interactions on Test set: 28534


In [51]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [52]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['item_id']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [53]:
from IPython.display import display, clear_output

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [54]:
# Constants
SEED = 42 # random seed for reproducibility
LR = 1e-3 # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.01 # lambda for L2 reg. ()
NUM_EPOCHS = 3 # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995 # learning rate scheduler parameter
BATCH_SIZE = 3000 # training batch size
EVAL_BATCH_SIZE = 3000 # evaluation batch size.
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # device to make the calculations on
ALPHA = 0.00001  # kl_divergence coefficient

In [55]:
total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())
total_df['user_id'], users_keys = total_df.user_id.factorize()
total_df['item_id'], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[:len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df):].values

/tmp/ipykernel_105786/627896225.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())


In [56]:
from scipy.sparse import csr_matrix
shape = [int(total_df['user_id'].max()+1), int(total_df['item_id'].max()+1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])), shape=shape).toarray()

In [57]:

# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__() # to initialize the parent class
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self): # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [58]:

# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(
    UserOrientedDataset(X_train),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dl = DataLoader(
    UserOrientedDataset(X_test),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

dls = {'train': train_dl, 'test': test_dl}

## Значимо изменить архитектуру модели

сделаем автоэнкодер вариационным

In [59]:
import torch
from torch import nn
from torch.nn import functional as F

In [60]:
class VariationalEncoder(nn.Module):
    def __init__(self, in_features=8287, latent_dims=512, device="cpu"):
        super(VariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(in_features, 1024)
        self.linear2 = nn.Linear(1024, latent_dims)
        self.linear3 = nn.Linear(1024, latent_dims)
        self.device = device

        self.N = torch.distributions.Normal(0, 1)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        mu = self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma * self.N.sample(mu.shape).to(self.device)
        return z, mu, sigma


class Decoder(nn.Module):
    def __init__(self, out_features=8287, latent_dims=512, device="cpu"):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 1024)
        self.linear2 = nn.Linear(1024, out_features)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = F.relu(self.linear2(z))
        return z


class AEModel(nn.Module):
    def __init__(self, in_out_features=8287, latent_dims=512, device="cpu"):
        super(AEModel, self).__init__()
        self.encoder = VariationalEncoder(in_out_features, latent_dims, device=device)
        self.decoder = Decoder(in_out_features, latent_dims)

    def forward(self, x):
        z, mu, sigma = self.encoder(x)
        return self.decoder(z), mu, sigma

In [61]:
def kl_divergence(mu, sigma):
    return (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()

In [62]:
torch.manual_seed(SEED) # Fix random seed to have reproducible weights of model layers

model = AEModel(device=DEVICE)
model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

def rmse_for_sparse(x_pred, x_true):
    mask = (x_true > 0)
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1/2)

In [63]:
# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
    "Train KL": [],
    "Test KL": [],
}

# Train loop
for epoch in range(NUM_EPOCHS):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'test']:
        with torch.set_grad_enabled(stage == 'train'): # Whether to start building a graph for a backward pass
            if stage == 'train':
                model.train()
            else:
                model.eval()

            rmse_loss_at_stage = 0
            kl_loss_at_stage = 0
            for batch in dls[stage]:
                batch = batch.to(DEVICE)
                x_pred, mu, sigma = model(batch) # forward pass: model(x_batch) -> calls forward()
                loss_rmse = rmse_for_sparse(x_pred, batch) # ¡Important! y_pred is always the first arg
                loss_kl = kl_divergence(mu, sigma)*ALPHA
                loss = loss_rmse + loss_kl
                if stage == "train":
                    loss.backward() # Calculate the gradients of all the parameters wrt loss
                    optimizer.step() # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad() # Zero the saved gradient
                rmse_loss_at_stage += loss_rmse.item() * len(batch)
                kl_loss_at_stage += loss_kl.item() * len(batch)
            rmse_at_stage = (rmse_loss_at_stage / len(dls[stage].dataset)) ** (1/2)
            kl_at_stage = (kl_loss_at_stage / len(dls[stage].dataset))
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)
            metrics_dict[f"{stage.title()} KL"].append(kl_at_stage)

    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE,Train KL,Test KL
0,0,2.269746,2.160939,6.565169,6.121464
1,1,2.023833,1.811274,5.938448,6.233556
2,2,1.599579,1.598038,5.862396,6.106135


In [64]:
with torch.no_grad():
    X_pred_test, mu, sigma = model(torch.Tensor(X_test).to(DEVICE))
X_pred_test

tensor([[7.7938, 7.2975, 6.0647,  ..., 0.0000, 0.0000, 0.6055],
        [5.3298, 5.0117, 3.8516,  ..., 0.0000, 0.0000, 0.4923],
        [3.2810, 3.1787, 2.5923,  ..., 0.0000, 0.0000, 0.5277],
        ...,
        [5.8217, 5.6679, 5.0780,  ..., 0.0000, 0.0000, 0.0000],
        [4.3294, 3.4890, 3.3209,  ..., 0.0290, 0.0000, 0.4684],
        [5.8075, 5.2872, 4.9076,  ..., 0.0000, 0.0000, 0.5176]])

In [77]:
class AERecommender:
    
    MODEL_NAME = 'Autoencoder'
    
    def __init__(self, X_preds, X_train_and_val, X_test):

        self.X_preds = X_preds.cpu().detach().numpy()
        self.X_train_and_val = X_train_and_val
        self.X_test = X_test
        
    def get_model_name(self):
        return self.MODEL_NAME

    def one_recomend(self, user_id, topn=10, verbose=False):
        X_total = self.X_train_and_val + self.X_test
        all_nonzero = np.argwhere(X_total[user_id] > 0).ravel()
        select_from = np.setdiff1d(np.arange(X_total.shape[1]), all_nonzero)

        user_preds = self.X_preds[user_id][select_from]
        items_idx = select_from[np.argsort(-user_preds)[:topn]]

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        return items_idx

    def recommend_items(self, user_id, items_to_select_idx, topn=10, verbose=False):
        user_preds = self.X_preds[user_id][items_to_select_idx]
        items_idx = items_to_select_idx[np.argsort(-user_preds)[:topn]]

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        return items_idx

    def evaluate(self, size=100):

        X_total = self.X_train_and_val + self.X_test

        true_5 = []
        true_10 = []

        for user_id in range(len(X_test)):
            non_zero = np.argwhere(self.X_test[user_id] > 0).ravel()
            all_nonzero = np.argwhere(X_total[user_id] > 0).ravel()
            select_from = np.setdiff1d(np.arange(X_total.shape[1]), all_nonzero)

            for non_zero_idx in non_zero:
                random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False)
                preds = self.recommend_items(user_id, np.append(random_non_interacted_100_items, non_zero_idx), topn=10)
                true_5.append(non_zero_idx in preds[:5])
                true_10.append(non_zero_idx in preds)

        return {"recall@5": np.mean(true_5), "recall@10": np.mean(true_10)}
    
ae_recommender_model = AERecommender(X_pred_test, X_train, X_train)

In [76]:
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': 0.8128370405765162, 'recall@10': 0.9360585877695106}

In [67]:
DEVICE='cpu'
model.to(DEVICE)
model.eval()
torch.save(model.state_dict(), '../models/torch_aencoder.ckpt')

In [78]:
ae_recommender_model.one_recomend(42)

array([821, 244, 179,   4,   6, 760, 938,  42,  72, 763])

In [81]:
import dill

with open('../models/autoencoder.dill', 'wb') as model_file:
    dill.dump(ae_recommender_model, model_file)